In [1]:
import re, os, sys
from functools import reduce
from itertools import combinations as comb, permutations as perm, combinations_with_replacement as combr
from operator import itemgetter
from pprint import pprint
from math import *
from collections import defaultdict
import networkx as nx
import numpy as np

def dprint(d):
    pprint(dict(d))
def lmap(*args, **kwargs):
    return list(map(*args, **kwargs))
def lmapi(*args, **kwargs):
    return list(map(int, *args, **kwargs))
def enum(*args, **kwargs):
    return enumerate(*args, **kwargs)

In [48]:
input_fname = "../inputs/day21-demo.in"

In [112]:
input_fname = "../inputs/day21.in"

In [113]:
def reread():
    lines = []
    with open(input_fname) as ifile:
        def r(): return ifile.readline().strip()
        def rs(): return ifile.readline().strip().split()
        def ri(): return int(ifile.readline().strip())
        def ris(): return list(map(int, ifile.readline().strip().split()))

        d = nx.DiGraph()
        for line in ifile:
            name, operation = line.strip().split(": ")
            if operation.isdigit():
                d.add_node(name, const=int(operation))
            else:
                m1, op, m2 = operation.split()
                d.add_node(name, op=operation)
                d.add_edge(m1, name)
                d.add_edge(m2, name)

    return d

d = reread()
list(d.nodes)[:5]

['qfzv', 'stgm', 'nzpf', 'djpv', 'psrp']

In [ ]:
dict(d.nodes(data=True))

## Part 1

In [33]:
d.nodes["root"]["op"]

'hsdb + mwrd'

In [50]:
val = dict(d.nodes(data="const"))
for node in nx.topological_sort(d):
    if val[node] is None:
        result = eval(d.nodes[node]["op"], None, val)
        val[node] = result
        

In [51]:
val["root"]

152.0

## Part 2

In [36]:
d.nodes["root"]

{'op': 'hsdb + mwrd'}

In [54]:
"humn" in nx.ancestors(d, "pppw")

True

In [53]:
"humn" in nx.ancestors(d, "sjmn")

False

In [43]:
val["mwrd"]

34588563455325.0

In [83]:
d.subgraph("sjmn drzm dbpl hmdt zczc")

In [102]:
def evaluate(d, root):
    subd = d.subgraph(list(nx.ancestors(d, root)) + [root])
    val = dict(subd.nodes(data="const"))
    for node in nx.topological_sort(subd):
        if val[node] is None:
            result = eval(subd.nodes[node]["op"], None, val)
            val[node] = result
            # print(node, "=", result)
        if node == root:
            return val[node]
    print("no result found for", root, subd.nodes, val)

In [103]:
evaluate(d, "sjmn")

150

In [119]:
cur = "root"
humn = "humn"
left, _, right = d.nodes[cur]["op"].split()
if humn in nx.ancestors(d, left):
    cur = left
    target = evaluate(d, right)
else:
    cur = right
    target = evaluate(d, left)
while True:
    # print("at node", cur, "target", target)
    left, op, right = d.nodes[cur]["op"].split()
    if humn in nx.ancestors(d, left):
        cur = left
        ready = right
    else:
        cur = right
        ready = left

    if op == "+":
        newtarget = target - evaluate(d, ready)
    elif op == "*":
        newtarget = target / evaluate(d, ready)
    elif op == "-":
        if ready == left:  # target = ready - newtarget
            newtarget = evaluate(d, ready) - target
        else:  # target = newtarget - ready
            newtarget = target + evaluate(d, ready)
    elif op == "/":
        if ready == left:  # target = ready / newtarget
            newtarget = evaluate(d, ready) / target
        else:  # target = newtarget / ready
            newtarget = target * evaluate(d, ready)
    
    target = newtarget
    if humn in d.nodes[cur]["op"]:
        print("humn child found for", cur, "=", d.nodes[cur]["op"])
        print("target", target)
        left, _, right = d.nodes[cur]["op"].split()
        if left == humn:
            print("other", right, evaluate(d, right))
        else:
            print("other", left, evaluate(d, left))
        break

humn child found for mnfl = cqgp + humn
target 3059361894462.0
other cqgp 542.0


In [117]:
3059361894462.0 - 542

3059361893920.0

In [47]:
val["hsdb"]

(90177205187826.53-18.17001180637544j)